In [1]:
import pandas as pd
import numpy as np

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
data = pd.read_csv('cs-training.csv', index_col=0)

In [3]:
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
X, y = data.drop('SeriousDlqin2yrs', axis=1).fillna(0), data.SeriousDlqin2yrs

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33)

In [5]:
class Prepare(object):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.scaler_ = StandardScaler()
        self.scaler_.fit(X, y)

    def transform(self, X, y, num_classes=2):
        X_r = self.scaler_.transform(X)
        y_r = keras.utils.to_categorical(y, num_classes)
        return X_r, y_r

In [21]:
X, y = data.drop('SeriousDlqin2yrs', axis=1).fillna(0), data.SeriousDlqin2yrs

X_train_raw, _, y_train_raw, _ = train_test_split(
    X, y, test_size=0.33)

prep = Prepare()
prep.fit(X_train_raw, y_train_raw)
X, y = prep.transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33)

del X_train_raw

In [7]:
X_train.shape

(100500, 10)

In [33]:
from collections import namedtuple
Layer = namedtuple('Layer', 'size dropout activation')

class NeuralNetwork(object):
    def __init__(self, 
                 layers=[Layer(512, 0.2, 'relu')],
                 batch_size=32, epochs=30,
                 optimizer='RMSprop',
                 n_classes=2,
                 n_predictors=10,
                 validation_set=None):
        assert validation_set is not None
        self.model_ = Sequential([
            Dense(layers[0].size, activation=layers[0].activation, 
                  input_shape=(n_predictors,)),
            Dropout(layers[0].dropout)])
        for layer in layers[1:]:
            self.model_.add(Dense(layer.size, activation=layer.activation))
            self.model_.add(Dropout(layer.dropout))
        self.model_.add(Dense(n_classes, activation='softmax'))
        self.model_.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.batch_size_ = batch_size
        self.epochs_ = epochs
        self.validation_set_ = validation_set
    
    def fit(self, X, y):
        self.history_ = self.model_.fit(
            X, y,
            batch_size=self.batch_size_,
            epochs=self.epochs_,
            verbose=1,
            validation_data=self.validation_set_)

    def predict_proba(self, X, batch_size=128, verbose=0):
        return self.model_.predict_proba(
            X, batch_size=batch_size, verbose=0)

In [37]:
valid_set = (X_test, y_test)
nn = NeuralNetwork(validation_set=valid_set)
nn.fit(X_train, y_train)

Train on 100500 samples, validate on 49500 samples
Epoch 1/30
100500/100500 [==============================] - 8s - loss: 0.2115 - acc: 0.9360 - val_loss: 0.2039 - val_acc: 0.9356
Epoch 2/30
100500/100500 [==============================] - 6s - loss: 0.2066 - acc: 0.9366 - val_loss: 0.2049 - val_acc: 0.9357
Epoch 3/30
100500/100500 [==============================] - 7s - loss: 0.2061 - acc: 0.9368 - val_loss: 0.2033 - val_acc: 0.9359
Epoch 4/30
100500/100500 [==============================] - 7s - loss: 0.2065 - acc: 0.9364 - val_loss: 0.2057 - val_acc: 0.9355
Epoch 5/30
100500/100500 [==============================] - 8s - loss: 0.2070 - acc: 0.9365 - val_loss: 0.2037 - val_acc: 0.9358
Epoch 6/30
100500/100500 [==============================] - 8s - loss: 0.2067 - acc: 0.9360 - val_loss: 0.2044 - val_acc: 0.9355
Epoch 7/30
100500/100500 [==============================] - 7s - loss: 0.2071 - acc: 0.9364 - val_loss: 0.2051 - val_acc: 0.9358
Epoch 8/30
100500/100500 [====================

KeyboardInterrupt: 

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from hyperopt.pyll.base import scope

def objective(params):
    print(params)
    l1 = Layer(params['units1'], params['dropout1'], params['activation'])
    l2 = Layer(params['units2'], params['dropout2'], params['activation'])
    layers = [l1, l2]
    choice = params['choice']
    if choice['layers']== 'three':
        l3 = Layer(choice['units3'], choice['dropout3'], params['activation'])
        layers.append(l3)   
        
    results = list()
    skf = StratifiedKFold(5)
    for train, test in skf.split(X_train, y_train_raw):
        X_train_inner = X_train[train]
        y_train_inner = y_train[train]
        X_test_inner = X_train[test]
        y_test_inner = y_train[test]
        valid_set = (X_test_inner, y_test_inner)
        nn = NeuralNetwork(
            layers=layers, validation_set=valid_set,
            epochs=params['epochs'],
            batch_size=params['batch_size'])
        nn.fit(X_train_inner, y_train_inner)
        pred_auc = nn.predict_proba(X_test_inner, batch_size=128, 
                                    verbose=0)
        auc = metrics.roc_auc_score(y_test_inner, pred_auc)
        results.append(auc)
    print('AUC:', np.mean(auc))
    return {'loss': -np.mean(auc), 'status': STATUS_OK}

space = {
    'choice': hp.choice('num_layers', [{
            'layers': 'two',
        }, {
            'layers': 'three',
            'units3': scope.int(hp.uniform('units3', 64, 1024)),
            'dropout3': hp.uniform('dropout3', .2, .75)
        }]),
    'units1': scope.int(hp.uniform('units1', 64, 1024)),
    'units2': scope.int(hp.uniform('units2', 64, 1024)),
    'dropout1': hp.uniform('dropout1', .25, .75),
    'dropout2': hp.uniform('dropout2', .25, .75),
    'batch_size': scope.int(hp.uniform('batch_size', 28, 128)),
    'epochs': 100,
    'optimizer': hp.choice('optimizer', ['adadelta', 'adam', 'rmsprop']),
    'activation': 'relu'
}

trials = Trials()
best = fmin(
    objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print(best)

{'activation': 'relu', 'batch_size': 57, 'choice': {'layers': 'two'}, 'dropout1': 0.2820765770716157, 'dropout2': 0.46782696417424885, 'epochs': 100, 'optimizer': 'adadelta', 'units1': 965, 'units2': 679}
Train on 80399 samples, validate on 20101 samples
Epoch 1/100
80399/80399 [==============================] - 5s - loss: 0.2187 - acc: 0.9356 - val_loss: 0.2015 - val_acc: 0.9373
Epoch 2/100
80399/80399 [==============================] - 5s - loss: 0.2129 - acc: 0.9363 - val_loss: 0.2046 - val_acc: 0.9382
Epoch 3/100
80399/80399 [==============================] - 4s - loss: 0.2140 - acc: 0.9364 - val_loss: 0.2053 - val_acc: 0.9372
Epoch 4/100
80399/80399 [==============================] - 4s - loss: 0.2145 - acc: 0.9362 - val_loss: 0.2063 - val_acc: 0.9376
Epoch 5/100
80399/80399 [==============================] - 4s - loss: 0.2145 - acc: 0.9359 - val_loss: 0.2073 - val_acc: 0.9372
Epoch 6/100
80399/80399 [==============================] - 4s - loss: 0.2153 - acc: 0.9362 - val_loss: 0.

80399/80399 [==============================] - 4s - loss: 0.2409 - acc: 0.9355 - val_loss: 0.2265 - val_acc: 0.9367
Epoch 63/100
80399/80399 [==============================] - 4s - loss: 0.2392 - acc: 0.9356 - val_loss: 0.2347 - val_acc: 0.9365
Epoch 64/100
80399/80399 [==============================] - 5s - loss: 0.2388 - acc: 0.9362 - val_loss: 0.2438 - val_acc: 0.9362
Epoch 65/100
80399/80399 [==============================] - 4s - loss: 0.2426 - acc: 0.9365 - val_loss: 0.2336 - val_acc: 0.9368
Epoch 66/100
80399/80399 [==============================] - 4s - loss: 0.2366 - acc: 0.9363 - val_loss: 0.2255 - val_acc: 0.9367
Epoch 67/100
80399/80399 [==============================] - 4s - loss: 0.2396 - acc: 0.9360 - val_loss: 0.2300 - val_acc: 0.9372
Epoch 68/100
80399/80399 [==============================] - 4s - loss: 0.2379 - acc: 0.9362 - val_loss: 0.2381 - val_acc: 0.9369
Epoch 69/100
80399/80399 [==============================] - 4s - loss: 0.2378 - acc: 0.9364 - val_loss: 0.2395

80399/80399 [==============================] - 4s - loss: 0.2213 - acc: 0.9364 - val_loss: 0.2154 - val_acc: 0.9362
Epoch 26/100
80399/80399 [==============================] - 4s - loss: 0.2250 - acc: 0.9358 - val_loss: 0.2143 - val_acc: 0.9357
Epoch 27/100
80399/80399 [==============================] - 4s - loss: 0.2214 - acc: 0.9359 - val_loss: 0.2179 - val_acc: 0.9360
Epoch 28/100
80399/80399 [==============================] - 4s - loss: 0.2211 - acc: 0.9360 - val_loss: 0.2141 - val_acc: 0.9360
Epoch 29/100
80399/80399 [==============================] - 4s - loss: 0.2232 - acc: 0.9362 - val_loss: 0.2182 - val_acc: 0.9358
Epoch 30/100
80399/80399 [==============================] - 4s - loss: 0.2223 - acc: 0.9365 - val_loss: 0.2151 - val_acc: 0.9362
Epoch 31/100
80399/80399 [==============================] - 4s - loss: 0.2229 - acc: 0.9363 - val_loss: 0.2195 - val_acc: 0.9355
Epoch 32/100
80399/80399 [==============================] - 4s - loss: 0.2241 - acc: 0.9359 - val_loss: 0.2161

80399/80399 [==============================] - 4s - loss: 0.2458 - acc: 0.9361 - val_loss: 0.2408 - val_acc: 0.9363
Epoch 89/100
80399/80399 [==============================] - 4s - loss: 0.2443 - acc: 0.9368 - val_loss: 0.2500 - val_acc: 0.9363
Epoch 90/100
80399/80399 [==============================] - 4s - loss: 0.2535 - acc: 0.9362 - val_loss: 0.2455 - val_acc: 0.9361
Epoch 91/100
80399/80399 [==============================] - 4s - loss: 0.2494 - acc: 0.9368 - val_loss: 0.2444 - val_acc: 0.9360
Epoch 92/100
80399/80399 [==============================] - 4s - loss: 0.2473 - acc: 0.9365 - val_loss: 0.2481 - val_acc: 0.9362
Epoch 93/100
80399/80399 [==============================] - 4s - loss: 0.2487 - acc: 0.9360 - val_loss: 0.2388 - val_acc: 0.9367
Epoch 94/100
80399/80399 [==============================] - 4s - loss: 0.2506 - acc: 0.9364 - val_loss: 0.2448 - val_acc: 0.9360
Epoch 95/100
80399/80399 [==============================] - 4s - loss: 0.2585 - acc: 0.9361 - val_loss: 0.2421

80400/80400 [==============================] - 4s - loss: 0.2202 - acc: 0.9366 - val_loss: 0.2345 - val_acc: 0.9325
Epoch 52/100
80400/80400 [==============================] - 4s - loss: 0.2239 - acc: 0.9367 - val_loss: 0.2300 - val_acc: 0.9324
Epoch 53/100
80400/80400 [==============================] - 4s - loss: 0.2261 - acc: 0.9361 - val_loss: 0.2241 - val_acc: 0.9325
Epoch 54/100
80400/80400 [==============================] - 4s - loss: 0.2209 - acc: 0.9366 - val_loss: 0.2219 - val_acc: 0.9328
Epoch 55/100
80400/80400 [==============================] - 4s - loss: 0.2228 - acc: 0.9366 - val_loss: 0.2247 - val_acc: 0.9318
Epoch 56/100
80400/80400 [==============================] - 4s - loss: 0.2236 - acc: 0.9361 - val_loss: 0.2266 - val_acc: 0.9332
Epoch 57/100
80400/80400 [==============================] - 4s - loss: 0.2251 - acc: 0.9361 - val_loss: 0.2265 - val_acc: 0.9322
Epoch 58/100
80400/80400 [==============================] - 4s - loss: 0.2306 - acc: 0.9361 - val_loss: 0.2245